In [1]:
from PIL import Image, ImageDraw
import random
import math

In [ ]:
def generate_arrow_image(
    width: int = 320,
    height: int = 320,
    min_length: float = 50,
    max_length: float = 150,
    base_angle_deg: float = 180,
    angle_variation_deg: float = 20,
    shaft_width: int = 3,
    head_length: float = 10,
    head_half_width: float = 5
) -> Image.Image:
    """
    Generate an image with a black arrow on a white background.
    The arrow's direction is randomly perturbed around base_angle_deg by ±angle_variation_deg.

    Args:
        width, height:  Canvas size in pixels.
        min_length, max_length:  Random range for arrow shaft length (in pixels).
        base_angle_deg:  Main arrow direction (in degrees), e.g., 180 = left.
        angle_variation_deg:  Angle variation range (in degrees).
        shaft_width:  Line width of the arrow shaft.
        head_length:  Length of the arrowhead (from tip backward).
        head_half_width:  Half-width of the arrowhead base.

    Returns:
        A PIL.Image.Image object with white background and a black arrow.
    """
    while True:
        # 1. Randomly determine arrow length L and direction angle θ (in radians)
        L = random.uniform(min_length, max_length)
        theta_deg = random.uniform(base_angle_deg - angle_variation_deg,
                                   base_angle_deg + angle_variation_deg)
        theta = math.radians(theta_deg)

        # 2. Compute direction vector (dx, dy) and unit vector (ux, uy)
        dx = L * math.cos(theta)
        dy = L * math.sin(theta)
        dist = math.hypot(dx, dy)
        ux = dx / dist
        uy = dy / dist

        # 3. Determine valid range for arrow tip (x2, y2)
        #    Requirements:
        #      a) 0 ≤ x2 ≤ width,  0 ≤ y2 ≤ height
        #      b) Arrow tail (x1 = x2 - dx, y1 = y2 - dy) also stays within canvas
        x2_min = max(0, dx)
        x2_max = min(width, width + dx)
        y2_min = max(0, dy)
        y2_max = min(height, height + dy)

        # Retry if the range is invalid
        if x2_min > x2_max or y2_min > y2_max:
            continue

        # 4. Randomly choose arrow tip (x2, y2) in valid region
        x2 = random.uniform(x2_min, x2_max)
        y2 = random.uniform(y2_min, y2_max)

        # 5. Compute arrow tail (x1, y1)
        x1 = x2 - dx
        y1 = y2 - dy

        # 6. Compute the triangle vertices of the arrowhead
        #    (1) Tip is (x2, y2)
        #    (2) Base center of the arrowhead
        base_x = x2 - head_length * ux
        base_y = y2 - head_length * uy

        #    (3) Perpendicular unit vector (px, py)
        px = -uy
        py = ux

        #    Left and right base corners of the arrowhead
        left_x  = base_x + head_half_width * px
        left_y  = base_y + head_half_width * py
        right_x = base_x - head_half_width * px
        right_y = base_y - head_half_width * py

        # 7. Check if all points lie within the canvas
        all_x = [x1, x2, left_x, right_x]
        all_y = [y1, y2, left_y, right_y]
        if (min(all_x) < 0 or max(all_x) > width or
            min(all_y) < 0 or max(all_y) > height):
            continue

        # If all constraints are satisfied, break and proceed to drawing
        break

    # 8. Create white canvas and draw the arrow
    img = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(img)

    # Draw the arrow shaft
    draw.line((x1, y1, x2, y2), fill='black', width=shaft_width)

    # Draw the arrowhead triangle
    draw.polygon([
        (x2, y2),
        (left_x, left_y),
        (right_x, right_y)
    ], fill='black')

    return img

In [ ]:
for i in range(5):
    img = generate_arrow_image(base_angle_deg=180, shaft_width= 5, head_length=20, head_half_width= 15)
    img.save(f'arrow_{i}.png')